In [0]:

file_location = "/FileStore/full_grouped_base_dataset_csv.csv"
file_type = "csv"

infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

#display(df)

temp_table_name = "full_grouped_base_dataset_csv"

df.createOrReplaceTempView(temp_table_name)

df.write.mode("overwrite").saveAsTable("full_grouped_base_dataset_csv")

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/tbsilvercoviddataset',True)

Out[101]: False

In [0]:
%sql
CREATE TABLE IF NOT EXISTS default.tbSilverCovidDataset
(
Data STRING
,Pais STRING
,Confirmados STRING
,Mortes STRING
,Recuperados STRING
,Ativos STRING
,Novos_casos STRING
,Novas_mortes STRING
,Novos_recuperados STRING
,Continente STRING
);

In [0]:
%sql
Insert into default.tbSilverCovidDataset
(
Data 
,Pais 
,Confirmados 
,Mortes 
,Recuperados 
,Ativos
,Novos_casos 
,Novas_mortes 
,Novos_recuperados 
,Continente 
)
select 
Date	
,Country_Region	
,Confirmed	
,Deaths	
,Recovered	
,Active	
,New_cases	
,New_deaths	
,New_recovered	
,WHO_Region
from default.full_grouped_base_dataset_csv

num_affected_rows,num_inserted_rows
33184,33184


In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/tbgoldcoviddataset',True)

Out[104]: False

In [0]:
%sql
CREATE TABLE IF NOT EXISTS default.tbGoldCovidDataset
(
Data STRING
,Pais STRING
,Confirmados STRING
,Mortes STRING
,Recuperados STRING
,Ativos STRING
,Novos_casos STRING
,Novas_mortes STRING
,Novos_recuperados STRING
,Continente STRING
);

In [0]:
%sql
Insert into default.tbGoldCovidDataset
(
Data 
,Pais 
,Confirmados 
,Mortes 
,Recuperados 
,Ativos
,Novos_casos 
,Novas_mortes 
,Novos_recuperados 
,Continente 
)
select 
Data 
,Pais 
,Confirmados 
,Mortes 
,Recuperados 
,Ativos
,Novos_casos 
,Novas_mortes 
,Novos_recuperados 
,Continente 
from default.tbSilverCovidDataset
where 
Confirmados > 0
or Mortes > 0
or Recuperados  > 0
or Ativos > 0
or Novos_casos  > 0
or Novas_mortes  > 0
or Novos_recuperados  > 0


num_affected_rows,num_inserted_rows
25166,25166


In [0]:
%sql
select month(data) Mes, sum(Novos_casos) Novos_casos
from default.tbGoldCovidDataset
group by month(data)
order by month(data) asc

Mes,Novos_casos
1,9372.0
2,75379.0
3,786064.0
4,2412383.0
5,2921042.0
6,4265801.0
7,3473901.0


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select month(data) Mes, sum(Novos_casos) Novos_casos,sum(Novas_mortes) Novas_mortes,sum(Novos_recuperados) Novos_recuperados
from default.tbGoldCovidDataset
group by month(data)
order by month(data) asc

Mes,Novos_casos,Novas_mortes,Novos_recuperados
1,9372.0,196.0,191.0
2,75379.0,2723.0,38095.0
3,786064.0,41542.0,135760.0
4,2412383.0,190226.0,815542.0
5,2921042.0,138902.0,1595973.0
6,4265801.0,137604.0,2695870.0
7,3473901.0,82573.0,2429906.0


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select  Pais, sum(Novos_casos) Novos_casos
from default.tbGoldCovidDataset
group by Pais
order by Novos_casos desc
LIMIT 10

Pais,Novos_casos
US,3576156.0
Brazil,2046328.0
India,1039084.0
Russia,751612.0
Peru,341586.0
Chile,326439.0
South Africa,324221.0
Mexico,324041.0
United Kingdom,294116.0
Iran,269440.0


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select  Pais, sum(Novos_casos) Novos_casos, sum(Novas_mortes) Novas_mortes , sum(Novos_recuperados) Novos_recuperados
from default.tbGoldCovidDataset
group by pais
order by Novos_casos desc
LIMIT 10;




Pais,Novos_casos,Novas_mortes,Novos_recuperados
US,3576156.0,138358.0,1090645.0
Brazil,2046328.0,77851.0,1428520.0
India,1039084.0,26273.0,653751.0
Russia,751612.0,11920.0,530801.0
Peru,341586.0,12615.0,230994.0
Chile,326439.0,8347.0,296814.0
South Africa,324221.0,4669.0,165591.0
Mexico,324041.0,37574.0,257681.0
United Kingdom,294116.0,45204.0,1403.0
Iran,269440.0,13791.0,232873.0


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
Select month(data),pais,sum(Novos_casos) Novos_casos, sum(Novas_mortes) Novas_mortes , sum(Novos_recuperados) Novos_recuperados from 
default.tbGoldCovidDataset
where Pais in (
            select Pais from 
                            (
                          select Pais, sum(Novos_casos) Novos_casos
                          from default.tbGoldCovidDataset
                          group by Pais
                          order by  Novos_casos desc
                          LIMIT 4
                            )
               ) 
group by month(data),pais
order by  month(data) desc

month(data),pais,Novos_casos,Novas_mortes,Novos_recuperados
7,India,453603.0,8873.0,305839.0
7,Russia,104683.0,2614.0,118828.0
7,Brazil,644287.0,18257.0,640202.0
7,US,939743.0,10926.0,370014.0
6,US,837290.0,22068.0,275873.0
6,Brazil,887192.0,30280.0,581763.0
6,India,394872.0,11992.0,256060.0
6,Russia,241086.0,4613.0,240090.0
5,Russia,299345.0,3620.0,160264.0
5,US,726457.0,41108.0,290811.0


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select  Continente, sum(Novos_casos) Novos_casos
from default.tbGoldCovidDataset
group by Continente
order by Novos_casos desc
LIMIT 10

Continente,Novos_casos
Americas,7376844.0
Europe,3073304.0
Eastern Mediterranean,1349193.0
South-East Asia,1348186.0
Africa,542799.0
Western Pacific,253616.0


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
select  month(data), sum(Novos_casos) Novos_casos, sum(Novas_mortes) Novas_mortes , sum(Novos_recuperados) Novos_recuperados
from default.tbGoldCovidDataset
where pais = 'Brazil'
group by month(data)
order by month(data)


month(data),Novos_casos,Novas_mortes,Novos_recuperados
2,2.0,0.0,0.0
3,5715.0,201.0,127.0
4,81470.0,5805.0,35808.0
5,427662.0,23308.0,170620.0
6,887192.0,30280.0,581763.0
7,644287.0,18257.0,640202.0


Databricks visualization. Run in Databricks to view.